In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qeji663d
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qeji663d
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-k6ofdsgm/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpz5va2mjw".


In [4]:
def generate_test_file(file_path, rows, columns):
    content = f""

    # Matrix 1 with 1s
    for index in range(rows):
        content += " ".join([str(float(index))] * columns) + "\n"


    file.write(content)

file_path = 'input_10m.txt'
n_cases = 1
sizes =[(10000000,1)]

with open(file_path, 'a') as file:

    for size in sizes:
        m,n = size
        generate_test_file(file_path, m, n)

print(f"Test file '{file_path}' generated successfully.")

Test file 'input_10m.txt' generated successfully.


**Question1 (Q1)**

In [5]:
%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>

using namespace std;

// Kernel definition
__global__ void ArrAdd(double *arr, long size,double *result)
{
    extern __shared__ double partialSum[];
    unsigned int t = threadIdx.x;

    unsigned int new_size = (size-1) / blockDim.x +1 ;
    partialSum[t] = 0;

    for(int i=0;i<new_size ;i++)
    {
      partialSum[t]+= arr[t*new_size+i];
    }

    if(t >=size)
      return ;
    for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

        if(t % (2*stride) == 0)
              partialSum[t] += partialSum[t+stride];
        __syncthreads();
    }

    if(t==0)
        *result = partialSum[0];
    }

int main(int argc, char *argv[])
{

    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    double* array = nullptr;  // Pointer to the array
    double * result = nullptr;
    double value;
    long count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // host to device allocation
     double *array_device;
     double *result_device;

     result = (double*)malloc(sizeof(double));
     cudaMalloc((void**)&array_device, count * sizeof(long));
     cudaMalloc((void**)&result_device,sizeof(double));

    cudaMemcpy(array_device, array, count * sizeof(long), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1, 1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(double) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(double), cudaMemcpyDeviceToHost);

    //cout<<*result<<endl;
    cout << fixed <<setprecision(2) << *result <<endl;

    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(result_device);

   return 0;
}


Writing Q1_2_31_2_33.cu


In [6]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1

In [7]:
!./Q1 "input_10.txt"

45.00


In [8]:
!./Q1 "input_100.txt"

4950.00


In [9]:
!./Q1 "input_1000.txt"

499500.00


In [10]:
!./Q1 "input_10th.txt"

49995000.00


In [11]:
!./Q1 "input_100th.txt"

4999950000.00


In [12]:
!./Q1 "input_1m.txt"

499999500000.00


In [13]:
!./Q1 "input_10m.txt"

49999995000000.00


In [14]:
!nvprof ./Q1 "input_1m.txt"

==1729== NVPROF is profiling process 1729, command: ./Q1 input_1m.txt
499999500000.00
==1729== Profiling application: ./Q1 input_1m.txt
==1729== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.78%  1.6026ms         1  1.6026ms  1.6026ms  1.6026ms  ArrAdd(double*, long, double*)
                   49.15%  1.5513ms         1  1.5513ms  1.5513ms  1.5513ms  [CUDA memcpy HtoD]
                    0.07%  2.0810us         1  2.0810us  2.0810us  2.0810us  [CUDA memcpy DtoH]
      API calls:   97.47%  162.72ms         2  81.360ms  72.620us  162.65ms  cudaMalloc
                    2.02%  3.3713ms         2  1.6856ms  1.6170ms  1.7543ms  cudaMemcpy
                    0.29%  485.57us         2  242.78us  105.01us  380.56us  cudaFree
                    0.12%  198.94us         1  198.94us  198.94us  198.94us  cudaLaunchKernel
                    0.09%  145.62us       114  1.2770us     146ns  52.122us  cuDeviceGetAttribute
 

In [15]:
!nvprof ./Q1 "input_10m.txt"

==1744== NVPROF is profiling process 1744, command: ./Q1 input_10m.txt
49999995000000.00
==1744== Profiling application: ./Q1 input_10m.txt
==1744== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.75%  18.552ms         1  18.552ms  18.552ms  18.552ms  [CUDA memcpy HtoD]
                   48.24%  17.293ms         1  17.293ms  17.293ms  17.293ms  ArrAdd(double*, long, double*)
                    0.01%  2.1440us         1  2.1440us  2.1440us  2.1440us  [CUDA memcpy DtoH]
      API calls:   83.01%  184.41ms         2  92.205ms  127.88us  184.28ms  cudaMalloc
                   16.25%  36.094ms         2  18.047ms  17.311ms  18.782ms  cudaMemcpy
                    0.58%  1.2776ms         2  638.82us  172.62us  1.1050ms  cudaFree
                    0.09%  208.49us         1  208.49us  208.49us  208.49us  cudaLaunchKernel
                    0.06%  134.24us       114  1.1770us     138ns  55.239us  cuDeviceGetAttribu

**Question2 (Q2)**

In [16]:
%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>

using namespace std;

// Kernel definition
__global__ void BinarySearch(double *array, long size, double target, long *index)
{

    __shared__ long shared_start;
    __shared__ long shared_end;
    __shared__ bool shared_found;
    __shared__ bool target_found;

    int tid = threadIdx.x;

    *index = -1;

    if (tid == 0) {
        shared_start = 0;
        shared_end = size - 1;
        shared_found = false;
        target_found = false;
    }
    __syncthreads();

    while (shared_start <= shared_end && shared_found == false && target_found == false) {

        long elements_per_thread = (shared_end - shared_start + 1 + blockDim.x - 1) / blockDim.x;
        long start = min(shared_start + tid * elements_per_thread, shared_end);
        long end = min(start + elements_per_thread - 1, shared_end);

        /*
        if (end > shared_end) {
            end = shared_end;
        }
        if(start > end){
          start = end;
        }*/


        if (array[start] <= target && target <= array[end]) {
            shared_start = start;
            shared_end = end;
            shared_found = true;
        }

        __syncthreads();

        if (shared_found == false && shared_start == shared_end){
          target_found = true;
          *index = -1;
        }
        else if (shared_found == true && shared_start == shared_end){
          target_found = true;
          *index = shared_start;
        }
        else if(shared_found == false){
           target_found = true;
           *index = -1;
        }
        shared_found = false;

    }
}


int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    double target = atof(argv[2]);

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

    double* array = nullptr;  // Pointer to the array
    long * result = nullptr;
    double value;
    long count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        double* temp = static_cast<double*>(realloc(array, (count + 1) * sizeof(double)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();


     // host to device allocation
     double *array_device;
     long *result_device;

     result = (long*)malloc(sizeof(long));

     cudaMalloc((void**)&array_device, count * sizeof(long));
     cudaMalloc((void**)&result_device,sizeof(long));

    cudaMemcpy(array_device, array, count * sizeof(double), cudaMemcpyHostToDevice);


    dim3 threadsPerBlock(1024, 1);
    dim3 numBlocks(1,1);

    BinarySearch<<< numBlocks, threadsPerBlock>>>(array_device,count, target,result_device);

    cudaMemcpy( result, result_device, sizeof(long), cudaMemcpyDeviceToHost);

    cout<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(result_device);

   return 0;
}


Writing Q2_2_31_2_33.cu


In [17]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2

In [18]:
!./Q2 "input_10.txt" 2.0

2


In [19]:
!./Q2 "input_10.txt" 8.0

8


In [20]:
!./Q2 "input_10.txt" 8.5

-1


In [21]:
!./Q2 "input_10.txt" -5

-1


In [22]:
!./Q2 "input_10.txt" 10.0

-1


In [23]:
!./Q2 "input_100.txt" 98.0

98


In [24]:
!./Q2 "input_100.txt" 55.0

55


In [25]:
!./Q2 "input_100.txt" 100.0

-1


In [26]:
!./Q2 "input_100.txt" -10

-1


In [27]:
!./Q2 "input_1000.txt" 300.0

300


In [28]:
!./Q2 "input_1000.txt" 999.0

999


In [29]:
!./Q2 "input_1000.txt" 1000.0

-1


In [30]:
!./Q2 "input_10th.txt" 5500.0

5500


In [31]:
!./Q2 "input_10th.txt" 9999.0

9999


In [32]:
!./Q2 "input_10th.txt" 10000.0

-1


In [33]:
!./Q2 "input_10th.txt" -66

-1


In [34]:
!./Q2 "input_100th.txt" 98765.0

98765


In [35]:
!./Q2 "input_100th.txt" 98765.6333

-1


In [36]:
!./Q2 "input_100th.txt" 100000.0

-1


In [37]:
!./Q2 "input_100th.txt" -10

-1


In [38]:
!./Q2 "input_1m.txt" 900000.0

900000


In [39]:
!./Q2 "input_1m.txt" 987654.0

987654


In [40]:
!./Q2 "input_1m.txt" 999999.0

999999


In [41]:
!./Q2 "input_1m.txt" 999999.563

-1


In [42]:
!./Q2 "input_1m.txt" 1000002.0

-1


In [43]:
!./Q2 "input_10m.txt" -10

-1


In [44]:
!./Q2 "input_10m.txt" 10000000.1

-1


In [45]:
!./Q2 "input_10m.txt" 8400000.56

-1


In [46]:
!./Q2 "input_10m.txt" 8400000.0

8400000


In [47]:
!nvprof ./Q2 "input_1m.txt" 999999.0

==1978== NVPROF is profiling process 1978, command: ./Q2 input_1m.txt 999999.0
999999
==1978== Profiling application: ./Q2 input_1m.txt 999999.0
==1978== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.04%  1.5497ms         1  1.5497ms  1.5497ms  1.5497ms  [CUDA memcpy HtoD]
                    0.82%  12.896us         1  12.896us  12.896us  12.896us  BinarySearch(double*, long, double, long*)
                    0.13%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   98.46%  166.13ms         2  83.065ms  72.939us  166.06ms  cudaMalloc
                    1.05%  1.7707ms         2  885.33us  24.123us  1.7465ms  cudaMemcpy
                    0.29%  494.47us         2  247.24us  111.62us  382.86us  cudaFree
                    0.12%  194.13us         1  194.13us  194.13us  194.13us  cudaLaunchKernel
                    0.07%  124.19us       114  1.0890us     134ns  48.734us  c

In [48]:
!nvprof ./Q2 "input_1m.txt" 999999.5

==1993== NVPROF is profiling process 1993, command: ./Q2 input_1m.txt 999999.5
-1
==1993== Profiling application: ./Q2 input_1m.txt 999999.5
==1993== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.29%  1.6388ms         1  1.6388ms  1.6388ms  1.6388ms  [CUDA memcpy HtoD]
                    0.59%  9.6960us         1  9.6960us  9.6960us  9.6960us  BinarySearch(double*, long, double, long*)
                    0.13%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   98.40%  165.88ms         2  82.940ms  77.271us  165.80ms  cudaMalloc
                    1.10%  1.8614ms         2  930.72us  23.037us  1.8384ms  cudaMemcpy
                    0.29%  493.03us         2  246.52us  107.94us  385.10us  cudaFree
                    0.12%  194.51us         1  194.51us  194.51us  194.51us  cudaLaunchKernel
                    0.07%  125.85us       114  1.1030us     134ns  49.911us  cuDev

In [49]:
!nvprof ./Q2 "input_10m.txt" 9999999.0

==2008== NVPROF is profiling process 2008, command: ./Q2 input_10m.txt 9999999.0
9999999
==2008== Profiling application: ./Q2 input_10m.txt 9999999.0
==2008== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.90%  16.796ms         1  16.796ms  16.796ms  16.796ms  [CUDA memcpy HtoD]
                    0.09%  14.912us         1  14.912us  14.912us  14.912us  BinarySearch(double*, long, double, long*)
                    0.01%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
      API calls:   91.05%  190.57ms         2  95.286ms  81.959us  190.49ms  cudaMalloc
                    8.14%  17.035ms         2  8.5174ms  24.961us  17.010ms  cudaMemcpy
                    0.61%  1.2718ms         2  635.91us  170.05us  1.1018ms  cudaFree
                    0.10%  207.61us         1  207.61us  207.61us  207.61us  cudaLaunchKernel
                    0.09%  189.93us       114  1.6660us     183ns  73.296

In [50]:
!nvprof ./Q2 "input_10m.txt" 9999999.5

==2027== NVPROF is profiling process 2027, command: ./Q2 input_10m.txt 9999999.5
-1
==2027== Profiling application: ./Q2 input_10m.txt 9999999.5
==2027== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.93%  16.860ms         1  16.860ms  16.860ms  16.860ms  [CUDA memcpy HtoD]
                    0.06%  10.271us         1  10.271us  10.271us  10.271us  BinarySearch(double*, long, double, long*)
                    0.01%  2.0790us         1  2.0790us  2.0790us  2.0790us  [CUDA memcpy DtoH]
      API calls:   90.88%  186.87ms         2  93.437ms  101.45us  186.77ms  cudaMalloc
                    8.32%  17.102ms         2  8.5511ms  25.138us  17.077ms  cudaMemcpy
                    0.62%  1.2713ms         2  635.65us  176.92us  1.0944ms  cudaFree
                    0.11%  231.48us         1  231.48us  231.48us  231.48us  cudaLaunchKernel
                    0.06%  130.78us       114  1.1470us     141ns  52.065us  c